<a href="https://colab.research.google.com/github/DaeSeokSong/NLP-Aengmu/blob/main/Aengmu_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [ERROR 4] Korean(Hangul) breaking phenomenon Solution on Colab

In [ ]:
""" 
ERROR 4. plot "Korean" breaking phenomenon
- Solution: Installing(↓) and setting(plt.rc('font', family='NanumBarunGothic')) Nanum font, after runtime restart
"""
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 70 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 4 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existen

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Morpheme analyzer
*   [Reference](https://soohee410.github.io/compare_tagger)
*   [Install](https://sanghyu.tistory.com/170)

In [ ]:
# okt, komoran, kkma
# install konlpy (okt, komoran, kkma)
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

# mecab (take a long time)
# set env
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

# install konlpy (mecab)
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

bash: line 8: fg: no job control
bash: line 11: fg: no job control
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  DEPRECATION: mecab-python was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. You can find discussion regarding this at https://github.com/pypa/pip/issues/8368.


# Error Solution

1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings' [(JVM)](https://gyulogs.tistory.com/130)
2. NameError: name 'Tagger' is not defined [(Mecab)](https://sosomemo.tistory.com/31)
3. ParserError: Error tokenizing data. C error [(Pandas)](https://mskim8717.tistory.com/82)
4. plot "Korean" breaking phenomenon on Colab [(Matplotlib)](https://teddylee777.github.io/colab/colab-korean)

In [ ]:
""" 
ERROR 1. TypeError: startJVM() got an unexpected keyword argument 'convertStrings'
- Solution: /usr/local/lib/python3.7/dist-packages/konlpy/jvm.py, 67 line (convertStrings=True) comments processing and save jvm.py before import pakage
"""

""" 
ERROR 2. NameError: name 'Tagger' is not defined 
- Solution: Execute mecab.sh script (under code excute)
"""
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# Import
*   [KoNLPy Reperence](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)



In [ ]:
# import for MechineLearning
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras import metrics, losses, callbacks
from tensorflow.keras.optimizers import Adam, Adagrad, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from gensim.models.keyedvectors import Word2VecKeyedVectors

# import Morpheme analyzer
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from konlpy.utils import pprint

# import etc
from google.colab import files
import time
import os
import os.path

# Change run location, 제출시에 제거
%cd /content/drive/My Drive/DeepLearning/PROJECT_AON
# Matplotlib set font on NanumBarunGothic
plt.rc('font', family='NanumBarunGothic')
# Numpy print set
np.set_printoptions(linewidth=200, precision=2)

/content/drive/My Drive/DeepLearning/PROJECT_AON


#Grobal Variable
*   [Concept Reference](https://reniew.github.io/25/)

In [ ]:
"""
ranking about time spent morpheme analyzing (The fastest is number one.)

1. Mecab
2. Komoran
3. Okt
4. Kkma

Mecab is faster than Kkma about 30~40 times
Mecab is faster than Okt about 10 times
Mecab is faster than Komoran about 5 times
"""

# Early versions use macab.
MECAB = Mecab()

# Learning target's name
AI_TARGET_NAME = "대석"

# Output size Or Time_step
OOT = 5

# Word2vec model
CORPUS = Word2VecKeyedVectors(vector_size=OOT)

# Dataset number
DN = 0

# Funtion

Data Prepare Functions

In [ ]:
# Functions that extract reply time (minute)
def extract_replytime(content) :
    idx_list1 = [i for i, value in enumerate(content) if value == ']']
    idx_list2 = [i for i, value in enumerate(content) if value == '[']

    end_idx = idx_list1[1]
    start_idx = idx_list2[1] + 1

    replytime = content[start_idx : end_idx]

    if replytime[:2] == "오전" :
        replytime = (int(replytime[2:replytime.index(":")]) * 60) + int(replytime[replytime.index(":") + 1 :])
    else :
        replytime = 60 * 12 + (int(replytime[2:replytime.index(":")]) * 60) + int(replytime[replytime.index(":") + 1 :])

    return replytime

# Functions that comparison about reply time
def compare_replytime(pre_text, cur_text) :
    pre_time = extract_replytime(pre_text)
    cur_time = extract_replytime(cur_text)

    result = False
    if cur_time - pre_time >= 10 : result = True

    return result

# Functions that extract Data about kakao talk's content
def extract_data(X_dataset, y_dataset, talk_list) :
    global AI_TARGET_NAME

    for talk in talk_list :
        tmp_X_respondents = []
        tmp_y_respondents = []
        for texts in talk.values :
            tmp_X_replys = []
            tmp_y_replys = []

            tmp_X_texts = []
            tmp_y_texts = []

            before_respondent = ""

            texts = texts.tolist()
            for idx, text in enumerate(texts) : 
                # Don't extract email and date
                if (not "저장한 날짜" in text) and (not "--------------" in text) and (not "@" in text) :
                    if not "]" in text : 
                        if before_respondent == AI_TARGET_NAME : tmp_y_texts.append(text)
                        else : tmp_X_texts.append(text)

                        continue
                    else : cur_respondent = text[1:text.index("]")]

                    if (":" in text) and (before_respondent != "") :
                        if compare_replytime(texts[idx - 1], text) or cur_respondent != before_respondent:
                            if len(tmp_X_texts) != 0 : tmp_X_replys.append(tmp_X_texts)
                            if len(tmp_y_texts) != 0 : tmp_y_replys.append(tmp_y_texts)
                            tmp_X_texts = []
                            tmp_y_texts = []

                    if AI_TARGET_NAME in text[1:text.index("]")] :
                        ptext = text[(text.rfind(']') + 2) : len(text)]
                        if ptext.find("http") == -1 and ptext != "사진" and ptext != "" : 
                            before_respondent = AI_TARGET_NAME
                            if ptext != '' : tmp_y_texts.append(ptext)
                    elif "]" in text :
                        ptext = text[(text.rfind(']') + 2) : len(text)]
                        if ptext.find("http") == -1 and ptext != "사진" and ptext != "" : 
                            before_respondent = cur_respondent
                            if ptext != '' : tmp_X_texts.append(ptext)

                    if idx == len(texts) - 1 :
                        if len(tmp_X_texts) != 0 : tmp_X_replys.append(tmp_X_texts)
                        if len(tmp_y_texts) != 0 : tmp_y_replys.append(tmp_y_texts)

            if len(tmp_X_replys) != 0 : tmp_X_respondents.append(tmp_X_replys)
            if len(tmp_y_replys) != 0 : tmp_y_respondents.append(tmp_y_replys)

        if len(tmp_X_respondents) != 0 : X_dataset.append(tmp_X_respondents)
        if len(tmp_y_respondents) != 0 : y_dataset.append(tmp_y_respondents)

# Functions that init reply's data type is list. so, change data type to str by this function
def unzip_replys(X_dataset, y_dataset) :
    # If don't use list, data type is str. this is caused by need just one word on word2vec
    for idx, respondent in enumerate(X_dataset) : 
        for i, reply in enumerate(respondent) :
            X_dataset[idx][i] = X_dataset[idx][i][0][0].split()

    for idx, respondent in enumerate(y_dataset) : 
        for i, reply in enumerate(respondent) :
            y_dataset[idx][i] = y_dataset[idx][i][0][0].split()

Data Pre-processing functions

In [ ]:
# Functions that auto calibrate spacing functions
def auto_spacing(X_dataset, y_dataset) :
    calibrate_spcaing(X_dataset)
    calibrate_spcaing(y_dataset)

def calibrate_spcaing(context_dataset) :
    global MECAB

    for order, respondent in enumerate(context_dataset) :
        for idx, texts in enumerate(respondent) :
            for repairIdx, text in enumerate(texts) :
                analyzedRes = MECAB.pos(text)
                for mecabR in analyzedRes :
                    # "MAG" == Adverb
                    if mecabR[1] == "MAG" :
                        try :
                            startIdx = text.index(mecabR[0])
                            if len(mecabR[0]) == 1 :
                                if text[startIdx + 1] != " " and text[startIdx + 1].isalnum() : 
                                    if text[startIdx + 2] != " " :
                                        text = text[:startIdx] + text[startIdx] + " " + text[startIdx + 1 :]
                                        context_dataset[order][idx][repairIdx] = text
                            else : 
                                if text[startIdx + len(mecabR[0])] != " " and text[startIdx + len(mecabR[0])].isalnum() : 
                                    if text[startIdx + len(mecabR[0]) + 1] != " " :
                                        text = text[:startIdx] + text[startIdx : startIdx + len(mecabR[0])] + " " + text[startIdx + len(mecabR[0]) :]
                                        context_dataset[order][idx][repairIdx] = text
                        except IndexError :
                            continue

# Functions that delete stopword in sentence
def stopword_filtering(X_dataset, y_dataset) :
    delete_stopword(X_dataset)
    delete_stopword(y_dataset)

def delete_stopword(word_dataset) :
    global MECAB

    stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','하다']
    for order, respondent in enumerate(word_dataset) :
        for idx, reply in enumerate(respondent) :
            for i, text in enumerate(reply) :
                tokenized_data = MECAB.morphs(text)
                tokenized_data = [word for word in tokenized_data if not word in stop_words]
                result_text = ''.join(tokenized_data)
                if result_text != '' : word_dataset[order][idx][i] = result_text

# Functions that divide X, y dataset in learning set
def combine_dataset(X_dataset, y_dataset) :
    global OOT

    X = []
    y = []
    for idx in range(0, DN) :
        tmp_X = np.empty(DN)
        tmp_y = np.empty(DN)
        X_size = len(X_dataset[idx])
        y_size = len(y_dataset[idx])

        if X_size > y_size : 
            tmp_X = X_dataset[idx][1 : y_size + 1].flatten()
            tmp_y = y_dataset[idx].flatten()
        elif y_size > X_size : 
            tmp_X = X_dataset[idx].flatten()
            tmp_y = y_dataset[idx][1 : X_size + 1].flatten()
        else :
            tmp_X = X_dataset[idx].flatten()
            tmp_y = y_dataset[idx].flatten()

        if idx == 0 :
            X.append(tmp_X)
            y.append(tmp_y)
        elif idx == 1 :
            # Type transformation list into ndarray
            X = np.r_[X[0], tmp_X]
            y = np.r_[y[0], tmp_y]
        else :
            X = np.r_[X, tmp_X]
            y = np.r_[y, tmp_y]

    # Set the total number of datasets to be a multiple of 10
    if len(X) % 10 != 0 : X = X[0 : (len(X) - (len(X) % 10 ))]
    if len(y) % 10 != 0 : y = y[0 : (len(y) - (len(y) % 10 ))]

    # Nan check
    if np.isnan(X.all()) : print("X dataset include NaN data")
    if np.isnan(y.all()) : print("y dataset include NaN data")

    # Reshape timestep and vector_length
    return X.reshape(int(X.shape[0]/OOT), OOT, 1), y.reshape(int(y.shape[0]/OOT), OOT, 1)

# 빈도수 너무 높거나 낮은건 제외하는 메서드 추가 고려 (데이터 학습 제대로 안 되면)

Word2Vec Functions (Distributed Representation)
*   [Word2Vec Reference 1](https://ebbnflow.tistory.com/153)
*   [Word2Vec Reference 2](https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/#%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec-%EB%A7%8C%EB%93%A4%EA%B8%B0)


*   [gensim Word2VecKeyedVectors, add](https://github.com/RaRe-Technologies/gensim/issues/2268)


*   [PCA(sklearn) Reference](https://m.blog.naver.com/tjdrud1323/221720259834)

In [ ]:
# Functions that draw a two-dimensional graph by entering the words, values of the two-dimensional X-axis, and values of the Y-axis.
def plot_2d_graph(vocabs, xs, ys):
    plt.figure(figsize=(8 ,6))
    plt.scatter(xs, ys, marker = 'o')
    for i, v in enumerate(vocabs):
        plt.annotate(v, xy=(xs[i], ys[i]))

# Functions that conversioning dataset to word2vec format
def dataset_to_word2vec(X_dataset, y_dataset) :
    return word2vec(X_dataset, True), word2vec(y_dataset, False)

def word2vec(word_dataset, isXdataset) :
    global CORPUS, OOT

    result = []
    for idx, respondent in enumerate(word_dataset) :
        # Init words and vectors
        w2v = Word2Vec(respondent, size=OOT, window=5, min_count=1, workers=6, sg=1)

        # Set word vectors
        if isXdataset : 
            CORPUS.add(entities=list(w2v.wv.vocab.keys()), weights=w2v.wv.vectors, replace=False)
        vocabs = CORPUS.vocab.keys()
        word_vector_list = [CORPUS[v] for v in vocabs]

        # Confirm word similarity
        # if "경호" in vocabs : print(word_vectors.most_similar("경호"))
        # else : print(word_vectors.most_similar("대석"))
        
        pca = PCA(n_components=OOT)
        xys = pca.fit_transform(word_vector_list)
        xs = xys[:,0]
        ys = xys[:,1]

        # Normalization
        scaler = MinMaxScaler()
        scaler.fit(xys)
        xys = scaler.transform(xys)

        # Change corpus's values to normalized data
        if isXdataset : 
            for idx, key in enumerate(vocabs) : CORPUS[[key]][0] = xys[idx]

        # plot_2d_graph(vocabs, xs, ys)
        result.append(xys.reshape(-1))

    return np.array(result)

def word2vec_talk(talk) :
    global OOT
    
    w2v = Word2Vec(talk, size=OOT, window=5, min_count=1, workers=6, sg=1)

    # Set word vectors
    word_vectors = w2v.wv
    vocabs = word_vectors.vocab.keys()
    word_vector_list = [word_vectors[v] for v in vocabs]
    
    word_vector_list = np.asarray(word_vector_list)

    # Confirm word similarity
    # if "경호" in vocabs : print(word_vectors.most_similar("경호"))
    # else : print(word_vectors.most_similar("대석"))

    return word_vector_list.reshape(-1)

# Functions that conversioning data that formed vector to word
def vec2word(vec, isreshape=False, isvec=True, beforeWords=[]) :
    global MECAB, CORPUS

    sentence = ""
    if isvec :
        if isreshape : sentence = sentence + CORPUS.most_similar(positive=[vec.reshape(OOT,)])[0][0]
        else : sentence = sentence + CORPUS.most_similar(positive=[vec])[0][0]
    else :
        word = CORPUS.most_similar(vec)[0][0]

        if not word in beforeWords : sentence = sentence + word
        else :
            for similar in CORPUS.most_similar(vec) :
                if not similar[0] in beforeWords :
                    sentence = sentence + similar[0]
                    break

    stop_poses = ['SF', 'SP', 'SS', 'SO', 'NF', 'NV', 'NA', 'EF', 'EC', 'EP']
    tokenized_data = MECAB.pos(sentence)
    for pos in stop_poses :
        if pos in tokenized_data[len(tokenized_data) - 1][1] : return sentence  

    if not sentence in beforeWords : beforeWords.append(sentence)
    sentence = sentence + ' ' + vec2word(sentence, isreshape=isreshape, isvec=False, beforeWords=beforeWords)
    
    return sentence

Model Learning Functions
*   [Keras loss function Reference](https://keras.io/api/losses/)

In [ ]:
# Functions that make up the model
def modeling_model() :
    global model

    model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding='same', input_shape=(OOT, 1)))
    model.add(MaxPooling1D(pool_size=3, padding='same'))
    model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding='same'))
    model.add(MaxPooling1D(pool_size=3, padding='same'))
    model.add(Conv1D(filters=64, kernel_size=3, activation="relu", padding='same'))
    model.add(MaxPooling1D(pool_size=3, padding='same'))
    model.add(Flatten())
    model.add(Dense(64, activation="sigmoid")) # Performance improves once it passes through layers that have been amplified datas
    model.add(Dropout(0.5))
    model.add(Dense(OOT,activation="softmax")) # output data == n( == y_dataset.shape[1]) things
    model.compile(
        loss=losses.MeanSquaredError(),
        optimizer=Adam(learning_rate=0.00001), # pram ex. learning_rate=0.0001
        metrics=[metrics.MeanSquaredError()]
    )

# Function to learn the configured model with the training datas
def training_model(X_train, y_train, X_val, y_val) :
    global model

    history = model.fit(X_train, y_train,
          validation_data = (X_val, y_val),
          batch_size = 20,
          epochs = 320,
          verbose = 1, # 0 = silent, 1 = progress bar,  2 = one line per epoch.
          )
    
    return history
    
# Function to evaluate the learned model
def evaluating_model(X_test, y_test, history) :
    global model

    # loss and acc graph (train and val)
    history_df = pd.DataFrame(history.history)
    history_df[["loss", "val_loss"]].plot()

    # Acc and Loss about real data
    learning_lost, learning_err = model.evaluate(X_test, y_test, verbose=2)
    print("Learning error % :", learning_err * 100)
    print("Learning loss % :", learning_lost * 100)

#Main

Prepare Data

In [ ]:
# uploaded_files = files.upload()

file_list = os.listdir('./')
fs = [f for f in file_list if f.endswith('.txt')]

talk_list = []
DN = 0
for f in fs :
    """ 
    ERROR 3. ParserError: Error tokenizing data. C error 
    - Solution: add code in read_csv = , sep='\t'
    """
    talk_list.append(pd.read_csv(f, sep='\t'))
    DN += 1

# Init Dataset
X_dataset = []
y_dataset = []

# Data extraction
extract_data(X_dataset, y_dataset, talk_list)

# Early version is used y_list[][0] and x_list[][0] (1 word) by learning model
unzip_replys(X_dataset, y_dataset)

Pre-processing Data
*  [Dataset Classification Reference 1](https://ganghee-lee.tistory.com/38)
*  [Dataset Classification Reference 2](https://ysyblog.tistory.com/69)

In [ ]:
# Space Modification
auto_spacing(X_dataset, y_dataset)

# Stopword deleted
stopword_filtering(X_dataset, y_dataset)

# Data type change on np.ndarray
X_dataset = np.asarray(X_dataset).astype(object)
y_dataset = np.asarray(y_dataset).astype(object)

# Word2Vec on dataset
X_dataset, y_dataset = dataset_to_word2vec(X_dataset, y_dataset)

# Set learning dataset
X_dataset, y_dataset = combine_dataset(X_dataset, y_dataset)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Model Training
*   [NLP Reference](https://towardsdatascience.com/natural-language-processing-a-crash-course-73d7a07c240c)
*   [Issue: Model predictional result is always same](https://github.com/keras-team/keras/issues/6447)

In [ ]:
# Data division (Train : Test = 8 : 2)
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, test_size = 0.2)
print("########## Train + Validation (X,) (y,) / Test (X,) (y,) ##########")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Data division (Train : Validation = 8 : 2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2)
print("########## Train (X,) (y,) / Validation (X,) (y,) ##########")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

print('\n')
if os.path.isfile('Aengmu.h5') : 
    # Load model
    print('#################### Model Load... ####################')
    model = tf.keras.models.load_model('Aengmu.h5')
else :
    print('#################### Init Model... ####################')
    # Laerning model
    model = Sequential()
    # Modeling
    modeling_model()
    
# Training
history = training_model(X_train, y_train, X_val, y_val)
# Evaluating
evaluating_model(X_test, y_test, history)

model.summary()

Prediction (Talk)

In [ ]:
while True :
    try :
        talk = input("앵무에게 보낼 카톡: ")

        if talk == "" : 
            print("대화 종료")
            break

        talk_vec = word2vec_talk([talk])
        talk_vec = talk_vec.reshape(int(talk_vec.shape[0]/OOT), OOT, 1)
        Aengmu = model.predict(talk_vec)

        print('앵무: ' + vec2word(Aengmu[len(Aengmu) - 1]) + '\n')
    except ValueError :
        print("ERROR, code 00001")
        break

앵무에게 보낼 카톡: this is test
앵무: 생각 만들고

앵무에게 보낼 카톡: 
대화 종료


Save Model

In [ ]:
#model.save('Aengmu.h5')